# Beginner: Retrieve TESS Data Validation Products with Astroquery

TESS (Transiting Exoplanet Survey Satelite) searches the 2-minute target data for transiting exoplanets. These data products are called DV (Data Validation) prodcuts. For every star in the search that produces a significant transiting event, a set of Data Validation products are created. The set of products are:

- DV reports (pdf), one per star
- DV summaries (pdf), one per transit signal found
- DV mini-reports (pdf), one per star
- DV time series files (fits),  one per star, one extension per transit signal

In this tutorial we show how to use astroquery to request all of the DV files available for a star of interest (L98-59 in this case). We then open the DV time series file to plot the detrended light curves produced by the mission and also a folded light curve for each signal found by the mission.

Skills explored in this notebook:

- Retrieving TESS timeseries observations with astroquery
- Retrieving TESS Data Validation products with astroquery
- Reading in a DV FITS file with astropy.fits
- Plotting with Matplotlib


---

## Input Statements

In [1]:
from astroquery.mast import Observations
from astroquery.mast import Catalogs
from astropy.io import fits
from astropy import table
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy

## Use Astroquery to find Observations of L98-59
We begin by doing a cone search usig the `Observations.query_object` function and then look at just those observations made by TESS that are time series observations. In this way we should get back just those that are TESS 2-minute cadence data observations. 

In [ ]:
star_name = "L98-59"

observations = Observations.query_object(star_name,radius = "0 deg")
obs_wanted = (observations['dataproduct_type'] == 'timeseries') & (observations['obs_collection'] == 'TESS')
print( observations[obs_wanted]['obs_collection', 'project','obs_id'] )

## Use Astroquery to dowload DV products
Use `Observations.get_product_list` and send in the observations of interest that we selected above to get the data proucts associated with those data products. Each observation contains many dataproducts associated with that observation. In this case we want those data products that have products subgroup description set to either DVT, DVM, DVS or DVR. The manifest is a table that contains the local path to the files that are downloaded.

In [ ]:
data_products = Observations.get_product_list(observations[obs_wanted])

files_wanted = (data_products["productSubGroupDescription"] == "DVT") | \
                (data_products["productSubGroupDescription"] == "DVM") | \
                (data_products["productSubGroupDescription"] == "DVS") | \
                (data_products["productSubGroupDescription"] == "DVR")

print(data_products["productFilename"][files_wanted])
manifest = Observations.download_products(data_products[files_wanted])


## Download Complete
You have now downloaded all the TESS DV products for this star and thier locations can be seen by printing the "Local Path" in the manifest.

In [ ]:
print( manifest['Local Path'] )

## Examine the download manifest

TESS file names tell you a lot about what is in the file. In the function `parse_manifest` below I break them apart so that I can make an easy to read table about the type of data that we downloaded.  Then we write out that part of the table.   This makes it obvious that there are lots of different sets of DV files based on different searches, each with a different sector range.  For each sector it was observed, there is a single sector search, and then there are also several multi sector searches.

In [ ]:
def parse_manifest(manifest):
    """
    Parse manifest and add back columns that are useful for TESS DV exploration.
    """
    results = deepcopy(manifest)
    filenames = []
    sector_range = []
    exts = []
    for i,f in enumerate(manifest['Local Path']):
        file_parts = np.array(np.unique(f.split(sep = '-')))
        sectors = list( map ( lambda x: x[0:2] == 's0', file_parts))
        s1 = file_parts[sectors][0]
        try:
            s2 = file_parts[sectors][1]
        except:
            s2 = s1
        sector_range.append("%s-%s" % (s1,s2))
        path_parts = np.array(f.split(sep = '/'))
        filenames.append(path_parts[-1])
        exts.append(path_parts[-1][-8:])

    results.add_column(table.Column(name = "filename", data = filenames))
    results.add_column(table.Column(name = "sectors", data = sector_range))
    results.add_column(table.Column(name = "fileType", data = exts))
    results.add_column(table.Column(name = "index", data = np.arange(0,len(manifest))))
    
    return results

#Run parser and print
results = parse_manifest(manifest)
print(results['index','sectors','fileType'])

## Plot the DVT File
The time series data use to find each transit signal (or Threshold Crossing event (TCE)) is found in the dvt.fits files.  As you can see there is a dvt file. If we want the file with the most data, we should pick the one with the longest sector range in the file name.  In this case it is s0001-s0013. 

In [ ]:
print(results['index', 'sectors', 'fileType'][results['sectors'] == "s0001-s0013"])

### Plot the entire DV median detrended, time series
The median detrended fluxes are stored in the first extension under 'LC_DETREND'. Notice in the plot before that L98-59, while in the continuous viewing zone, was not observed during every sector.

In [ ]:
# Locate the file that has the data
want = (results['sectors'] == "s0001-s0013") & (results['fileType'] == "dvt.fits")
dvt_filename = manifest[want]['Local Path'][0]

# Print out the file info
fits.info(dvt_filename)

# Plot the first time series
data = fits.getdata(dvt_filename, 1)
time = data['TIME']
relflux = data['LC_DETREND']

plt.figure(figsize=(16,3))
plt.plot (time, relflux, 'b.')
plt.ylim(1.2* np.nanpercentile(relflux, .5) , 1.2 * np.nanpercentile(relflux, 99.5))
plt.title('Data Validation Detrended Light Curve for %s' % (star_name))

### Plot Folded Light curve
Each extension of the DVT data file contains a separate TCE. After the pipeline finds a set of transits, the transits are removed and the light curve is once again searched for transits.  L98-59 has three TCEs, each is consistent with the three confirmed planets found around this star.  Here we plot the phase folded light curve for each TCE, each as its own subplot.  The DVT file also contains a transit model as one of the columns in the FITS table. We overplot that in orange.

In [ ]:
def plot_folded(phase, data, model, ext, period):
    isort = phase.argsort()
    
    plt.plot(phase[isort], data[isort], '.', ms = .5)
    plt.plot(phase[isort], model[isort], '-', lw=1, label = "TCE %i" % ext)
    plt.xlabel('Phase (Period = %5.2f days)' % period)
    plt.ylim(1.5 * np.nanpercentile(data, .5) , 1.4 * np.nanpercentile(data,99.5))
    plt.legend(loc = "lower right")

plt.figure(figsize = (14,10))

nTCEs = fits.getheader(dvt_filename)['NEXTEND'] - 2

for ext in range(1,nTCEs + 1):
    data = fits.getdata(dvt_filename, ext)
    head = fits.getheader(dvt_filename, ext)
    period = head['TPERIOD']
    phase = data['PHASE']
    flux = data['LC_INIT']
    model = data['MODEL_INIT']
    plt.subplot(3, 1, ext)
    plot_folded(phase, flux, model, ext, period)

<a id="about_ID"></a>
## About this Notebook
**Authors:** 
<br>Susan E. Mullally, STScI 
<br>**Updated On:** 2019-09-13

[Top of Page](#title_ID)
<img style="float: right;" src="./stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="stsci_pri_combo_mark_horizonal_white_bkgd" width="200px"/> 